# Machine Translation -  English to Spanish - Seq2Seq


#### Here we aim to create a english to spanish translation machine using encoder decoder models.

#### Data credits: http://www.manythings.org/anki/spa-eng.zip

In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import spacy

In [ ]:
NUM_SAMPLES = 10000 #Regulating the training set for faster training
MAX_VOCAB_SIZE = 20000
MAX_SEQ_LEN = 20
EMBEDDING_DIM = 300
LSTM_UNITS = 256

### Importing text

In [ ]:
path_spa = "spa.txt"

english_text = [] #used in encoder
spanish_text_in = [] #used in decoder
spanish_text_out = [] #used in decoder

In [ ]:
f = open(path_spa)
i = 0
for line in f:
    i+=1
    if(i>NUM_SAMPLES):
        break
    if '\t' in line:
        eng,spa,_ = line.split('\t')
        english_text.append(eng)
        spanish_text_in.append("<sos> "+spa)
        spanish_text_out.append(spa+" <eos>")

f.close()

**Printing some of the the appended texts**

In [ ]:
english_text[:3]

In [ ]:
spanish_text_in[:3]

In [ ]:
spanish_text_out[:3]

### Tokenizing the texts and Padding them

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Tokenizer for encoder**

In [ ]:
tokenizer_encoder = Tokenizer(num_words = MAX_VOCAB_SIZE)
tokenizer_encoder.fit_on_texts(english_text)
english_text = tokenizer_encoder.texts_to_sequences(english_text)

**Finding the maximum sequence length in english_text**

In [ ]:
max_seq_len_text = max(len(s) for s in english_text)
max_seq_len_text 

In [ ]:
max_seq_len_eng = min(max_seq_len_text,MAX_SEQ_LEN)
max_seq_len_eng

**Padding**

In [ ]:
english_text = pad_sequences(english_text,maxlen=max_seq_len_eng,padding='pre')
english_text[:3]

In [ ]:
encoder_vocab_size = min(MAX_VOCAB_SIZE,len(tokenizer_encoder.word_index)+1)
encoder_vocab_size

**Tokenizer for decoder**

In [ ]:
tokenizer_decoder = Tokenizer(num_words=MAX_VOCAB_SIZE,filters='')
tokenizer_decoder.fit_on_texts(["<eos>"] + spanish_text_in)
spanish_text_in = tokenizer_decoder.texts_to_sequences(spanish_text_in)
spanish_text_out = tokenizer_decoder.texts_to_sequences(spanish_text_out)

**Finding the maximum sequence length in spanish_text**

In [ ]:
max_seq_len_text = max(len(s) for s in spanish_text_in)
max_seq_len_text 

In [ ]:
max_seq_len_spa = min(max_seq_len_text,MAX_SEQ_LEN)
max_seq_len_spa 

**Padding**

In [ ]:
spanish_text_in = pad_sequences(spanish_text_in,maxlen=max_seq_len_spa,padding="post")
spanish_text_out = pad_sequences(spanish_text_out,maxlen=max_seq_len_spa,padding="post")

In [ ]:
spanish_text_out

In [ ]:
decoder_vocab_size = min(MAX_VOCAB_SIZE,len(tokenizer_decoder.word_index)+1)
decoder_vocab_size

### Creating Embedding matrix for Encoder (English Text)

In [ ]:
eng_word_index = tokenizer_encoder.word_index
eng_word_index

In [ ]:
spa_word_index = tokenizer_decoder.word_index
spa_word_index

In [ ]:
embedding_matrix = np.zeros((encoder_vocab_size,EMBEDDING_DIM))

In [ ]:
#loading spacy model for english language
nlp = spacy.load("en_core_web_lg")

In [ ]:
#generating embedding matrix 
for word,index in eng_word_index.items():
    if(index < encoder_vocab_size):
        vector = nlp(word).vector
        embedding_matrix[index] = vector

### One hot encoding the target sequence - spanish_text_out

In [ ]:
ohe_spanish_text_out = np.zeros((len(spanish_text_out),max_seq_len_spa,decoder_vocab_size))

for seq_i, seq in enumerate(spanish_text_out):
    for word_i,word in enumerate(seq):
        if(word>0):
            ohe_spanish_text_out[seq_i,word_i,word] = 1

### Creating the training model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM,Dense,Input,Embedding
from tensorflow.keras.utils import plot_model

**Encoder**

In [ ]:
encoder_input = Input(shape=(max_seq_len_eng))

In [ ]:
x = Embedding(encoder_vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],input_length = max_seq_len_eng)(encoder_input)

In [ ]:
lstm_enc = LSTM(LSTM_UNITS,activation='relu',return_state=True)

In [ ]:
enc_out,h,c = lstm_enc(x)

In [ ]:
encoder_states = [h,c]

**Decoder**

In [ ]:
decoder_input = Input(shape=(max_seq_len_spa))

In [ ]:
dec_embedding = Embedding(decoder_vocab_size,EMBEDDING_DIM,input_length=max_seq_len_spa)
x = dec_embedding(decoder_input)

In [ ]:
lstm_dec = LSTM(LSTM_UNITS,activation='relu',return_state=True)

In [ ]:
x,_,_ = lstm_dec(x,initial_state=encoder_states)

In [ ]:
dense_dec = Dense(decoder_vocab_size,activation="softmax")

In [ ]:
dec_out = dense_dec(x)

In [ ]:
model = Model([encoder_input,decoder_input],dec_out)

In [ ]:
model.summary()

In [ ]:
plot_model(model)

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#model.fit([english_text,spanish_text_in],ohe_spanish_text_out)

### Creating the sampling model

**Encoder**

In [ ]:
s_enc_model = Model(encoder_input,encoder_states)

In [ ]:
plot_model(s_enc_model)

**Decoder**

In [ ]:
s_decoder_input = Input(shape=(1))
s_decoder_input_h = Input(shape=(LSTM_UNITS))
s_decoder_input_c = Input(shape=(LSTM_UNITS))

s_decoder_states = [s_decoder_input_h,s_decoder_input_c]

In [ ]:
x = dec_embedding(s_decoder_input)

In [ ]:
x,h,c = lstm_dec(x,initial_state=s_decoder_states)

In [ ]:
s_decoder_out = dense_dec(x)

In [ ]:
s_dec_model = Model(inputs=[s_decoder_input]+s_decoder_states,outputs=[s_decoder_out] + [h,c])

#### Defining the translate function

In [ ]:
#reverse indexing required to convert token indices to words
spa_index_word = { s:v for v,s in spa_word_index.items()}

In [ ]:
def translate(english_sentence):
    output_spa = []
    
    x = tokenizer_encoder.texts_to_sequences([english_sentence])
    x = pad_sequences(x,maxlen=max_seq_len_eng)
    
    h,c = s_enc_model.predict(x)
    next_states = [h,c]
    
    next_word = np.array([spa_word_index['<sos>']])
    
    for i in range(max_seq_len_spa):
        next_word, h,c = s_dec_model.predict([next_word] + next_states)
        
        next_word = np.array([np.argmax(next_word)])
        next_states = [h,c]
        
        append_word = spa_index_word[next_word[0]]
        if(append_word == "<eos>"):
            break
            
        output_spa.append(append_word)
        
    return " ".join(output_spa)

In [ ]:
translate("hi good morning")